# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_key import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,uzhgorod,48.6167,22.3000,8.61,78,100,3.72,UA,1699781223
1,1,grytviken,-54.2811,-36.5092,2.63,97,55,1.17,GS,1699781224
2,2,port-aux-francais,-49.3500,70.2167,3.53,87,100,15.45,TF,1699781225
3,3,vila velha,-20.3297,-40.2925,24.41,88,0,5.66,BR,1699781225
4,4,tsiombe,-25.3000,45.4833,33.55,25,0,3.44,MG,1699781226


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE
city_map = city_data_df.hvplot.points(
    'Lng',
    'Lat',
    geo=True, 
    tiles = 'OSM', 
    frame_width = 700, 
    frame_height = 500, 
    color='City',
    alpha= 0.5,
    s='Humidity',
    scale= 0.8)
# Display the map
# YOUR CODE HERE
city_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather_df = city_data_df.loc[(city_data_df['Max Temp']<=25)\
                            &(city_data_df['Max Temp']>=0)\
                            &(city_data_df['Cloudiness']<=80)\
                            &(city_data_df['Country']!='US')\
                            &(city_data_df['Country']!='AU')\
                            &(city_data_df['Country']!='IN')\
                            &(city_data_df['Country']!='RU')\
                            &(city_data_df['Country']!='CN')\
                            &(city_data_df['Humidity']<50),:]
print(len(ideal_weather_df))

# Drop any rows with null values
ideal_weather_df.dropna()

# Display sample data
ideal_weather_df.head()

25


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
37,37,gaiman,-43.2897,-65.4929,6.11,41,44,13.40,AR,1699781241
53,53,necochea,-38.5473,-58.7368,14.56,45,77,9.44,AR,1699781249
103,103,sao joao do piaui,-8.3581,-42.2467,24.22,45,12,1.53,BR,1699781270
109,109,pokhara,28.2333,83.9833,22.94,46,0,1.85,NP,1699781060
122,122,vernon,50.2581,-119.2691,8.34,45,36,1.54,CA,1699781278


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name']=''

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
37,gaiman,AR,-43.2897,-65.4929,41,
53,necochea,AR,-38.5473,-58.7368,45,
103,sao joao do piaui,BR,-8.3581,-42.2467,45,
109,pokhara,NP,28.2333,83.9833,46,
122,vernon,CA,50.2581,-119.2691,45,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {'categories':'accommodation.hotel',\
         'filter':'',\
         'bias':'',\
         'apiKey':geoapify_key,\
         'limit':1}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lon = row['Lng']
    lat = row['Lat']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f'circle:{lon},{lat},{radius}'
    params["bias"] = f'proximity:{lon},{lat}'
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
gaiman - nearest hotel: yr hen ffordd
necochea - nearest hotel: Cabañas Río Quequén
sao joao do piaui - nearest hotel: Hotel Esplanada
pokhara - nearest hotel: Hotel Annapurna View Sarangkot
vernon - nearest hotel: Okanagan Royal Park Inn
serhetabat - nearest hotel: No hotel found
bafq - nearest hotel: هتل رستوران علمدار
sougueur - nearest hotel: Hôtel Bab Essahra
adrar - nearest hotel: No hotel found
ghat - nearest hotel: فندق تاسيلي
berriane - nearest hotel: No hotel found
muynoq - nearest hotel: Lala
constantia - nearest hotel: Hotel Chérica
cassia - nearest hotel: Hotel Âncora
general acha - nearest hotel: Hotel Patagonia
sampacho - nearest hotel: No hotel found
queenstown - nearest hotel: Queens Hotel
nushki - nearest hotel: No hotel found
khash - nearest hotel: No hotel found
chirchiq - nearest hotel: Кимёгар
turkestan - nearest hotel: Атамекен
plaza huincul - nearest hotel: Antu Malal
kentau - nearest hotel: No hotel found
kaka - nearest hotel: No hotel fou

,City,Country,Lat,Lng,Humidity,Hotel Name
37,gaiman,AR,-43.2897,-65.4929,41,yr hen ffordd
53,necochea,AR,-38.5473,-58.7368,45,Cabañas Río Quequén
103,sao joao do piaui,BR,-8.3581,-42.2467,45,Hotel Esplanada
109,pokhara,NP,28.2333,83.9833,46,Hotel Annapurna View Sarangkot
122,vernon,CA,50.2581,-119.2691,45,Okanagan Royal Park Inn
149,serhetabat,TM,35.2799,62.3438,34,No hotel found
157,bafq,IR,31.6128,55.4107,18,هتل رستوران علمدار
167,sougueur,DZ,35.1857,1.4961,45,Hôtel Bab Essahra
185,adrar,MR,20.5022,-10.0711,14,No hotel found
224,ghat,LY,24.9647,10.1728,24,فندق تاسيلي


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points('Lng',
            'Lat',
            geo=True, 
            tiles = 'OSM', 
            frame_width = 700, 
            frame_height = 500, 
            color='City',
            alpha=0.6,
            hover=True,
            hover_cols='all',
            use_index=False,
            s='Humidity',
            scale= 1.2)

# Display the map
hotel_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)